<a href="https://colab.research.google.com/github/oliviaperryman/colab-notebooks/blob/master/wav_phone_pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import IPython
import matplotlib
import matplotlib.pyplot as plt
import requests
import torch
import torchaudio

matplotlib.rcParams["figure.figsize"] = [16.0, 4.8]

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.__version__)
print(torchaudio.__version__)
print(device)

1.12.1+cu113
0.12.1+cu113
cpu


In [ ]:
!sudo apt-get install sox

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 20 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrnb0 amd64 0.1.3-2.1 [92.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopencore-amrwb0 amd64 0.1.3-2.1 [45.8 kB]
Get:3 http://archive.ubuntu.com

In [ ]:
! pip install audiosegment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 66 kB 2.9 MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp37-cp37m-linux_x86_64.whl size=72476 sha256=83805d169dccf70c8d10cfdaa76adec68b33727917484f33007ef83bbaaad2d4
  Stored in directory: /root/.cache/pip/wheels/11/f9/67/a3158d131f57e1c0a7d8d966a707d4a2fb27567a4fe47723ad
Successfully built webrtcvad


In [ ]:
import audiosegment
import pandas as pd

In [ ]:
seg = audiosegment.from_file("sa2.wav")


In [ ]:
seg

sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 2.157s long

In [ ]:
txt = open("sa2.txt", "rb")
phones = pd.read_csv("sa2.phn", header=None, delimiter=' ')


In [ ]:
phones

,0,1,2
0,0,2260,h#
1,2260,2730,d
2,2730,4120,uh
3,4120,4600,n
4,4600,6864,ae
5,6864,7920,s
6,7920,8270,kcl
7,8270,8640,k
8,8640,8856,m
9,8856,9520,ix


Timit sampled at 16000Hz

16000Hz to ms = 0.0625


In [ ]:
print(len(seg)) # millseconds in segment
print(34400 * 0.0625) # phones length converted to milliseconds

2157
2150.0


In [ ]:
seg[100:110]

sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.010s long

In [ ]:
from collections import defaultdict
phone_wavs = defaultdict(list)

In [ ]:
for _,(start,stop,phone) in phones.iterrows():
  snippet = seg[start* 0.0625 // 1:stop* 0.0625 //1]
  phone_wavs[phone].append(snippet)


In [ ]:
phone_wavs

defaultdict(list,
            {'h#': [sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.141s long,
              sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.080s long],
             'd': [sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.029s long],
             'uh': [sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.087s long],
             'n': [sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.030s long,
              sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.042s long],
             'ae': [sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.142s long,
              sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.165s long,
              sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.174s long],
             's': [sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.066s long],
             'kcl': [sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.021s long,
              sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.047s long,
              sa2.wav: 1

In [ ]:
phone_wavs.pop('h#')

[sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.141s long,
 sa2.wav: 1 channels, 16 bit, sampled @ 16.0 kHz, 0.080s long]

In [ ]:
import os
os.makedirs('paired_data')

In [ ]:
for phone, samples in phone_wavs.items():
  try:
    os.makedirs(f'paired_data/{phone}')
    for i,s in enumerate(samples):
      s.export(f'paired_data/{phone}/{phone}_{i}.wav', format='WAV')
  except Exception as e:
    print(e)

In [ ]:
!zip -r /content/paired_data.zip /content/paired_data

  adding: content/paired_data/ (stored 0%)
  adding: content/paired_data/tcl/ (stored 0%)
  adding: content/paired_data/tcl/tcl_0.wav (deflated 51%)
  adding: content/paired_data/s/ (stored 0%)
  adding: content/paired_data/s/s_0.wav (deflated 24%)
  adding: content/paired_data/m/ (stored 0%)
  adding: content/paired_data/m/m_0.wav (deflated 16%)
  adding: content/paired_data/eh/ (stored 0%)
  adding: content/paired_data/eh/eh_0.wav (deflated 15%)
  adding: content/paired_data/g/ (stored 0%)
  adding: content/paired_data/g/g_0.wav (deflated 32%)
  adding: content/paired_data/dh/ (stored 0%)
  adding: content/paired_data/dh/dh_0.wav (deflated 31%)
  adding: content/paired_data/k/ (stored 0%)
  adding: content/paired_data/k/k_1.wav (deflated 28%)
  adding: content/paired_data/k/k_0.wav (deflated 33%)
  adding: content/paired_data/n/ (stored 0%)
  adding: content/paired_data/n/n_1.wav (deflated 27%)
  adding: content/paired_data/n/n_0.wav (deflated 20%)
  adding: content/paired_data/dx/ (

# Get Embeddings


In [ ]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H

In [ ]:
model = bundle.get_model().to(device)

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth


  0%|          | 0.00/360M [00:00<?, ?B/s]

In [ ]:
waveform, sample_rate = torchaudio.load("paired_data/ae/ae_1.wav")

In [ ]:
waveform = waveform.to(device)

In [ ]:
# Both 16000
# if sample_rate != bundle.sample_rate:
#     waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

In [ ]:
waveform

tensor([[-1.1292e-03, -6.1035e-04,  9.1553e-05,  ...,  1.4038e-03,
          9.7656e-04,  1.0376e-03]])

In [ ]:
with torch.inference_mode():
    features, _ = model.extract_features(waveform)

In [ ]:
from glob import glob
all_wav_snippets = [y for x in os.walk("paired_data") for y in glob(os.path.join(x[0], '*.wav'))]

In [ ]:
feature_pairs = []

In [ ]:
with torch.inference_mode():
  for snippet in all_wav_snippets:
    try:
      waveform,  _  = torchaudio.load(snippet)
      waveform = waveform.to(device)
      features, _ = model.extract_features(waveform)
      feature_pairs.append(((snippet.split('/')[-1].split('_')[0]), features[-1]))
    except Exception as e:
      print(e)
      print(snippet)

Calculated padded input size per channel: (1). Kernel size: (2). Kernel size can't be greater than actual input size
paired_data/m/m_0.wav
Calculated padded input size per channel: (1). Kernel size: (2). Kernel size can't be greater than actual input size
paired_data/k/k_0.wav
Calculated padded input size per channel: (1). Kernel size: (2). Kernel size can't be greater than actual input size
paired_data/kcl/kcl_0.wav


In [ ]:
for fp in feature_pairs:
  print(fp[0])
  print(fp[1].shape)


tcl
torch.Size([1, 2, 768])
s
torch.Size([1, 3, 768])
eh
torch.Size([1, 2, 768])
g
torch.Size([1, 1, 768])
dh
torch.Size([1, 1, 768])
k
torch.Size([1, 3, 768])
n
torch.Size([1, 1, 768])
n
torch.Size([1, 1, 768])
dx
torch.Size([1, 1, 768])
oy
torch.Size([1, 8, 768])
oy
torch.Size([1, 4, 768])
uh
torch.Size([1, 4, 768])
ix
torch.Size([1, 3, 768])
ix
torch.Size([1, 2, 768])
ix
torch.Size([1, 1, 768])
d
torch.Size([1, 1, 768])
kcl
torch.Size([1, 5, 768])
kcl
torch.Size([1, 2, 768])
l
torch.Size([1, 2, 768])
ax
torch.Size([1, 2, 768])
r
torch.Size([1, 3, 768])
r
torch.Size([1, 3, 768])
ae
torch.Size([1, 8, 768])
ae
torch.Size([1, 6, 768])
ae
torch.Size([1, 8, 768])
gcl
torch.Size([1, 1, 768])


In [ ]:
import torch.nn.functional as F

In [ ]:
def zero_pad_tensor(tensor, dim=8):
  tensor = tensor.squeeze(dim=0)

  # pad if size is less than desired dim
  if tensor.shape[0] < dim:
    dims_to_add = dim - (tensor.shape[0])
    tensor = F.pad(tensor,(0,0,0,dims_to_add),"constant", 0)

  #truncate if size is greater than desired dim
  if tensor.shape[0] > dim:
    tensor = tensor[0:dim, :]

  return tensor

In [ ]:
for fp in feature_pairs:
  print(fp[0])
  a = fp[1]
  # print(a.shape)
  # a = a.squeeze(dim=0)
  # print(a.shape)
  # dims_to_add = 8 - (a.shape[0])
  # print(dims_to_add)
  # out = F.pad(a,(0,0,0,dims_to_add),"constant", 0)
  # out = a[0:5, :]
  out = zero_pad_tensor(a,8)
  print(out.shape)
  # print(out)


tcl
torch.Size([8, 768])
s
torch.Size([8, 768])
eh
torch.Size([8, 768])
g
torch.Size([8, 768])
dh
torch.Size([8, 768])
k
torch.Size([8, 768])
n
torch.Size([8, 768])
n
torch.Size([8, 768])
dx
torch.Size([8, 768])
oy
torch.Size([8, 768])
oy
torch.Size([8, 768])
uh
torch.Size([8, 768])
ix
torch.Size([8, 768])
ix
torch.Size([8, 768])
ix
torch.Size([8, 768])
d
torch.Size([8, 768])
kcl
torch.Size([8, 768])
kcl
torch.Size([8, 768])
l
torch.Size([8, 768])
ax
torch.Size([8, 768])
r
torch.Size([8, 768])
r
torch.Size([8, 768])
ae
torch.Size([8, 768])
ae
torch.Size([8, 768])
ae
torch.Size([8, 768])
gcl
torch.Size([8, 768])


In [ ]:
avg_feature_pairs = []

In [ ]:
for fp in feature_pairs:
  a = fp[1].squeeze()
  if len(a.shape) > 1:
    a = torch.mean(a,0)
  avg_feature_pairs.append((fp[0],a))

In [ ]:
import csv

In [ ]:
with open('vectors.tsv', 'w', newline='') as tsvfile:
  writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
  for _, vector in avg_feature_pairs:
    writer.writerow(vector.numpy())


In [ ]:
with open('labels.tsv', 'w', newline='') as tsvfile:
  writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
  for label, _ in avg_feature_pairs:
    writer.writerow([label])
